In [36]:
from collections import Counter
import pandas as pd
import math
import numpy as np 

We downloaded the "Hurricane" csv from the excel file mentioned in the question . Now we can apply changes as asked in the questions . 

* PART - A

In this part of the question we need to write program for t-test for correlation coefficient with a 1% level of significance between "“Max. sustained winds(mph)” and “Minimum pressure(mbar)”.

In [37]:
dataset = pd.read_csv('Assignment-1 - Hurricane.csv')

wind_speed = dataset['Max. sustained winds(mph)'] ;  

min_pressure = dataset['Minimum pressure(mbar)'] ; 

wind_speed = np.array(wind_speed) ; 
min_pressure = np.array(min_pressure) ; 
# print(wind_speed.shape) ; 
# print(wind_speed) ; 
# print(min_pressure.shape) ;  
# print(min_pressure) ;  


# Now we have got the column values in the np arrays . Now we can proceed with the t_test ; 




We will be using t_test to conduct t-test for correlation coefficient between “Max. sustained winds(mph)” and “Minimum pressure(mbar)” with 1% level of significance . Thus we formulate the hypothesis first : 

### Null Hypothesis (H₀)
There is no significant correlation between the two variables.

- H₀: ρ = 0

### Alternative Hypothesis (H₁)
There is a significant correlation between the two variables (two-tailed test).

- H₁: ρ ≠ 0

Let us first calculate the value of r (Pearson Correlation Coefficient) . 

In [38]:
# let r be the correlation coefficient . 

def calculate_mean(data) : 
  if (len(data) == 0):
    return None ; 
  
  return sum(data) / len(data) ;  


def calculate_num(wind_speed , min_pressure): 
  mean_wind_speed = calculate_mean(wind_speed) ; 
  
  mean_min_pressure = calculate_mean(min_pressure) ; 
  
  ans = 0 ; 
  
  for i in range(len(wind_speed)): 
    
    first_term = (wind_speed[i] - mean_wind_speed) ; 
    second_term = (min_pressure[i] -mean_min_pressure) ; 
    
    ans += first_term * second_term ; 
  
  return ans ; 

def calculate_den(wind_speed , min_pressure):
  mean_wind_speed = calculate_mean(wind_speed) ; 
  
  mean_min_pressure = calculate_mean(min_pressure) ; 
  
  ans = 0 ; 
  
  first_term = 0 ; 
  second_term = 0 ; 
  
  for i in range(len(wind_speed)):
    first_term += (wind_speed[i] - mean_wind_speed) ** 2 ; 
  
  for i in range(len(min_pressure)):
    second_term += (min_pressure[i] - mean_min_pressure) ** 2 ; 
  
  
  ans = first_term * second_term ; 
  
  ans = ans ** (1/2) ; 
  
  return ans ; 

numerator = calculate_num(wind_speed, min_pressure)
denominator = calculate_den(wind_speed, min_pressure)

if denominator != 0:  # To avoid division by zero
    r = numerator / denominator
else:
    r = None


print("The value of r is : " + str(r)) 


den_term = math.sqrt(1 - r * r) ; 

n = len(wind_speed) ; 

t = (r / den_term) * (math.sqrt(n - 2)) ; 

degree_of_freedom = (n - 2) ; 

print("The degree of freedom is: " + str(degree_of_freedom)) ; 
print("Value of t comes out to be: " + str(t)) ;  # we get the t-value from here . 

# Thus from the t-table shared to us , we can see that the value of alpha = 0.01 .(as the significance is 1% and the test is two tailed) . The value for n = 90 , cutoff = 2.632 and for n = 100 , cutoff = 2.626 . 

# Thus we know -5.49 < -2.632 , Thus we will reject the null hypothesis . 


The value of r is : -0.48359549563566406
The degree of freedom is: 99
Value of t comes out to be: -5.497270157344813


Thus from the t-table shared to us , we can see that the value of alpha = 0.01 .(as the significance is 1% and the test is two tailed) . The value for n = 90 , cutoff = 2.632 and for n = 100 , cutoff = 2.626 . 

Thus we know -5.49 < -2.632 , Thus we will reject the null hypothesis . 

# Part B

In this we will first change the categorical (Non-Numerical)